In [ ]:
#data
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
from detection_XAI.utils import utils
import cv2
from tensorflow.keras.models import load_model

x_cluster = io.loadmat('./SSL/data/x_cluster.mat')
x_cluster = x_cluster['x_cluster']

# labells
y_cluster = io.loadmat('./SSL/data/y_cluster.mat')
y_cluster = y_cluster['y_cluster']

#pre-processing
from keras.utils import to_categorical

y_cluster_n = to_categorical(y_cluster-1)
x_cluster_n = utils.norml(x_cluster)

x_cluster_stft = utils.batch_stft(x_cluster_n,nfft=128,overlap=0.7)

#reshape input to 64x64
def resize_stft(x_in):
  batch_num = x_in.shape[0]

  # Initialize an empty array to store resized images
  resized_input = np.zeros((batch_num, 64, 64, 3), dtype=np.float32)

  # Loop through each image in the batch and resize
  for i in range(batch_num):
      # Resize each image to (64, 64)
      resized_image = cv2.resize(x_in[i], (64, 64), interpolation=cv2.INTER_LINEAR)

      # Store the resized image in the new array
      resized_input[i] = resized_image


# Now 'resized_images' contains all the images resized to (64, 64, 3) with dtype float32
  return resized_input

resized_x = resize_stft(x_cluster_stft)

saved_encoder_model = load_model("./SSL/model/encoder_4x4x16_v2.h5")
encoded_features = saved_encoder_model.predict(resized_x)
reshaped_features = encoded_features.reshape(encoded_features.shape[0], -1)

43/43 [==============================] - 2s 8ms/step


In [ ]:
# train the model
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# Define the model architecture
def create_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(4, activation='softmax')  # Assuming 4 classes
    ])
    return model

# Create the model
model = create_model(input_shape=(256,))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_79"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_316 (Dense)           (None, 128)               32896     
                                                                 
 dropout_237 (Dropout)       (None, 128)               0         
                                                                 
 dense_317 (Dense)           (None, 64)                8256      
                                                                 
 dropout_238 (Dropout)       (None, 64)                0         
                                                                 
 dense_318 (Dense)           (None, 32)                2080      
                                                                 
 dropout_239 (Dropout)       (None, 32)                0         
                                                                 
 dense_319 (Dense)           (None, 4)               

In [ ]:
# Convert labels to one-hot encoded matrix
y_train_one_hot = to_categorical(y_train - 1)  # Subtract 1 because indices start from 0
y_test_one_hot = to_categorical(y_test - 1)
# Train the model
model.fit(X_train, y_train_one_hot, epochs=200, batch_size=128, validation_data=(X_test, y_test_one_hot), verbose=1)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

def resultshow(classes,label):
  lis=np.zeros(classes.shape[0])
  for i in range(classes.shape[0]):
   jeg = np.where(classes[i,:] == max(classes[i,:]))
   lis[i] = jeg[0]

  tru=np.zeros(classes.shape[0])
  for i in range(classes.shape[0]):
    jeg2 = np.where(label[i,:] == max(label[i,:]))
    tru[i] = jeg2[0]

  target = ['quake','earthquake','rockfall','enviroemnt noise']
  print(classification_report(tru, lis, target_names=target))
  print(confusion_matrix(tru, lis))
  confusion_matrices = (confusion_matrix(tru, lis))
  precision, recall, f1, support = precision_recall_fscore_support(tru, lis)
  return confusion_matrices, precision, recall, f1

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# K-fold validation (3-fold in this example)
kfold = StratifiedKFold(n_splits=3, shuffle=True)
data = reshaped_features
labels = y_cluster.flatten()

class_names = ['quake', 'earthquake', 'rockfall', 'noise']
precision_dicts = {class_name: [] for class_name in class_names}
recall_dicts = {class_name: [] for class_name in class_names}
f1_dicts = {class_name: [] for class_name in class_names}
confusion_matrices_list = []
accuracy_list = []

for train_index, test_index in kfold.split(data, labels):
    X_test, X_train = data[train_index], data[test_index]
    y_test, y_train = labels[train_index], labels[test_index]
    y_train_one_hot = to_categorical(y_train - 1)  # Subtract 1 because indices start from 0
    y_test_one_hot = to_categorical(y_test - 1)

    model = create_model(input_shape=(256,))
    # Compile the model
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(X_train, y_train_one_hot, epochs=200, batch_size=128, validation_data=(X_test, y_test_one_hot), verbose=0)

    results = model.predict(X_test)
    confusion_matrices, precision, recall, f1 = resultshow(results,y_test_one_hot)
    confusion_matrices_list.append(confusion_matrices)

    # Calculate accuracy for the current fold
    _, accuracy = model.evaluate(X_test, y_test_one_hot, verbose=0)
    accuracy_list.append(accuracy)

            # Update dictionaries for each class
    for class_name in class_names:
        class_index = class_names.index(class_name)
        precision_dicts[class_name].append(precision[class_index])
        recall_dicts[class_name].append(recall[class_index])
        f1_dicts[class_name].append(f1[class_index])

# Calculate mean and std for each class
for class_name in class_names:
    mean_precision = np.mean(precision_dicts[class_name])
    std_precision = np.std(precision_dicts[class_name])

    mean_recall = np.mean(recall_dicts[class_name])
    std_recall = np.std(recall_dicts[class_name])

    mean_f1 = np.mean(f1_dicts[class_name])
    std_f1 = np.std(f1_dicts[class_name])

    # Print mean and std for each class
    print(f"Class {class_name} - Mean Precision: {mean_precision:.3f} ± {std_precision:.3f}")
    print(f"Class {class_name} - Mean Recall: {mean_recall:.3f} ± {std_recall:.3f}")
    print(f"Class {class_name} - Mean F1-score: {mean_f1:.3f} ± {std_f1:.3f}")

# Calculate and print mean and std for accuracy
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)

print(f"Mean Accuracy: {mean_accuracy:.3f} ± {std_accuracy:.3f}")

29/29 [==============================] - 0s 2ms/step
                  precision    recall  f1-score   support

           quake       0.81      0.71      0.75       156
      earthquake       0.95      0.93      0.94       259
        rockfall       0.94      0.92      0.93       268
enviroemnt noise       0.80      0.91      0.85       233

        accuracy                           0.88       916
       macro avg       0.87      0.86      0.87       916
    weighted avg       0.88      0.88      0.88       916

[[110   5   6  35]
 [  8 240   4   7]
 [  8   3 246  11]
 [ 10   5   7 211]]


<ipython-input-24-0dae81680718>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  lis[i] = jeg[0]
<ipython-input-24-0dae81680718>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tru[i] = jeg2[0]


29/29 [==============================] - 0s 3ms/step
                  precision    recall  f1-score   support

           quake       0.75      0.81      0.78       156
      earthquake       0.95      0.91      0.93       259
        rockfall       0.90      0.93      0.91       268
enviroemnt noise       0.87      0.82      0.85       234

        accuracy                           0.88       917
       macro avg       0.87      0.87      0.87       917
    weighted avg       0.88      0.88      0.88       917

[[126   5   7  18]
 [ 10 236  11   2]
 [  6   4 250   8]
 [ 26   4  11 193]]


<ipython-input-24-0dae81680718>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  lis[i] = jeg[0]
<ipython-input-24-0dae81680718>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tru[i] = jeg2[0]


29/29 [==============================] - 0s 3ms/step
                  precision    recall  f1-score   support

           quake       0.77      0.81      0.79       156
      earthquake       0.90      0.92      0.91       260
        rockfall       0.86      0.92      0.89       268
enviroemnt noise       0.91      0.78      0.84       233

        accuracy                           0.86       917
       macro avg       0.86      0.86      0.86       917
    weighted avg       0.87      0.86      0.86       917

[[126  10   6  14]
 [  7 240  12   1]
 [  6  13 246   3]
 [ 24   5  23 181]]


<ipython-input-24-0dae81680718>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  lis[i] = jeg[0]
<ipython-input-24-0dae81680718>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tru[i] = jeg2[0]


Class quake - Mean Precision: 0.777 ± 0.024
Class quake - Mean Recall: 0.774 ± 0.048
Class quake - Mean F1-score: 0.774 ± 0.015
Class earthquake - Mean Precision: 0.931 ± 0.025
Class earthquake - Mean Recall: 0.920 ± 0.007
Class earthquake - Mean F1-score: 0.925 ± 0.012
Class rockfall - Mean Precision: 0.896 ± 0.032
Class rockfall - Mean Recall: 0.923 ± 0.007
Class rockfall - Mean F1-score: 0.909 ± 0.017
Class noise - Mean Precision: 0.861 ± 0.046
Class noise - Mean Recall: 0.836 ± 0.053
Class noise - Mean F1-score: 0.845 ± 0.005
Mean Accuracy: 0.875 ± 0.007


In [ ]:
confusion_matrices_array = np.array(confusion_matrices_list)

mean_confusion_matrix = np.mean(confusion_matrices_array, axis=0)
std_confusion_matrix = np.std(confusion_matrices_array, axis=0)


print("Mean Confusion Matrix:")
print(mean_confusion_matrix)
print("\nStandard Deviation Confusion Matrix:")
print(std_confusion_matrix)

Mean Confusion Matrix:
[[120.66666667   6.66666667   6.33333333  22.33333333]
 [  8.33333333 238.66666667   9.           3.33333333]
 [  6.66666667   6.66666667 247.33333333   7.33333333]
 [ 20.           4.66666667  13.66666667 195.        ]]

Standard Deviation Confusion Matrix:
[[ 7.54247233  2.3570226   0.47140452  9.10433352]
 [ 1.24721913  1.88561808  3.55902608  2.62466929]
 [ 0.94280904  4.49691252  1.88561808  3.29983165]
 [ 7.11805217  0.47140452  6.79869268 12.32882801]]
